<a href="https://colab.research.google.com/github/vrangayyan6/GenAI/blob/main/Deep_research_LangGraph_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Check out https://github.com/google-gemini/gemini-fullstack-langgraph-quickstart/tree/main
*   Create API key at https://ai.google.dev/

In [1]:
!pip install -q langgraph langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
# @title
# configuration.py

import os
from pydantic import BaseModel, Field
from typing import Any, Optional

from langchain_core.runnables import RunnableConfig


class Configuration(BaseModel):
    """The configuration for the agent."""

    query_generator_model: str = Field(
        default="gemini-2.0-flash",
        metadata={
            "description": "The name of the language model to use for the agent's query generation."
        },
    )

    reflection_model: str = Field(
        default="gemini-2.5-flash-preview-05-20",
        metadata={
            "description": "The name of the language model to use for the agent's reflection."
        },
    )

    answer_model: str = Field(
        default="gemini-2.5-flash-preview-05-20",                     # "gemini-2.5-pro-preview-05-06",
        metadata={
            "description": "The name of the language model to use for the agent's answer."
        },
    )

    number_of_initial_queries: int = Field(
        default=3,
        metadata={"description": "The number of initial search queries to generate."},
    )

    max_research_loops: int = Field(
        default=2,
        metadata={"description": "The maximum number of research loops to perform."},
    )

    @classmethod
    def from_runnable_config(
        cls, config: Optional[RunnableConfig] = None
    ) -> "Configuration":
        """Create a Configuration instance from a RunnableConfig."""
        configurable = (
            config["configurable"] if config and "configurable" in config else {}
        )

        # Get raw values from environment or config
        raw_values: dict[str, Any] = {
            name: os.environ.get(name.upper(), configurable.get(name))
            for name in cls.model_fields.keys()
        }

        # Filter out None values
        values = {k: v for k, v in raw_values.items() if v is not None}

        return cls(**values)

In [3]:
# @title
# utils.py

from typing import Any, Dict, List
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage


def get_research_topic(messages: List[AnyMessage]) -> str:
    """
    Get the research topic from the messages.
    """
    # check if request has a history and combine the messages into a single string
    if len(messages) == 1:
        research_topic = messages[-1].content
    else:
        research_topic = ""
        for message in messages:
            if isinstance(message, HumanMessage):
                research_topic += f"User: {message.content}\n"
            elif isinstance(message, AIMessage):
                research_topic += f"Assistant: {message.content}\n"
    return research_topic


def resolve_urls(urls_to_resolve: List[Any], id: int) -> Dict[str, str]:
    """
    Create a map of the vertex ai search urls (very long) to a short url with a unique id for each url.
    Ensures each original URL gets a consistent shortened form while maintaining uniqueness.
    """
    prefix = f"https://vertexaisearch.cloud.google.com/id/"
    urls = [site.web.uri for site in urls_to_resolve]

    # Create a dictionary that maps each unique URL to its first occurrence index
    resolved_map = {}
    for idx, url in enumerate(urls):
        if url not in resolved_map:
            resolved_map[url] = f"{prefix}{id}-{idx}"

    return resolved_map


def insert_citation_markers(text, citations_list):
    """
    Inserts citation markers into a text string based on start and end indices.

    Args:
        text (str): The original text string.
        citations_list (list): A list of dictionaries, where each dictionary
                               contains 'start_index', 'end_index', and
                               'segment_string' (the marker to insert).
                               Indices are assumed to be for the original text.

    Returns:
        str: The text with citation markers inserted.
    """
    # Sort citations by end_index in descending order.
    # If end_index is the same, secondary sort by start_index descending.
    # This ensures that insertions at the end of the string don't affect
    # the indices of earlier parts of the string that still need to be processed.
    sorted_citations = sorted(
        citations_list, key=lambda c: (c["end_index"], c["start_index"]), reverse=True
    )

    modified_text = text
    for citation_info in sorted_citations:
        # These indices refer to positions in the *original* text,
        # but since we iterate from the end, they remain valid for insertion
        # relative to the parts of the string already processed.
        end_idx = citation_info["end_index"]
        marker_to_insert = ""
        for segment in citation_info["segments"]:
            marker_to_insert += f" [{segment['label']}]({segment['short_url']})"
        # Insert the citation marker at the original end_idx position
        modified_text = (
            modified_text[:end_idx] + marker_to_insert + modified_text[end_idx:]
        )

    return modified_text


def get_citations(response, resolved_urls_map):
    """
    Extracts and formats citation information from a Gemini model's response.

    This function processes the grounding metadata provided in the response to
    construct a list of citation objects. Each citation object includes the
    start and end indices of the text segment it refers to, and a string
    containing formatted markdown links to the supporting web chunks.

    Args:
        response: The response object from the Gemini model, expected to have
                  a structure including `candidates[0].grounding_metadata`.
                  It also relies on a `resolved_map` being available in its
                  scope to map chunk URIs to resolved URLs.

    Returns:
        list: A list of dictionaries, where each dictionary represents a citation
              and has the following keys:
              - "start_index" (int): The starting character index of the cited
                                     segment in the original text. Defaults to 0
                                     if not specified.
              - "end_index" (int): The character index immediately after the
                                   end of the cited segment (exclusive).
              - "segments" (list[str]): A list of individual markdown-formatted
                                        links for each grounding chunk.
              - "segment_string" (str): A concatenated string of all markdown-
                                        formatted links for the citation.
              Returns an empty list if no valid candidates or grounding supports
              are found, or if essential data is missing.
    """
    citations = []

    # Ensure response and necessary nested structures are present
    if not response or not response.candidates:
        return citations

    candidate = response.candidates[0]
    if (
        not hasattr(candidate, "grounding_metadata")
        or not candidate.grounding_metadata
        or not hasattr(candidate.grounding_metadata, "grounding_supports")
    ):
        return citations

    for support in candidate.grounding_metadata.grounding_supports:
        citation = {}

        # Ensure segment information is present
        if not hasattr(support, "segment") or support.segment is None:
            continue  # Skip this support if segment info is missing

        start_index = (
            support.segment.start_index
            if support.segment.start_index is not None
            else 0
        )

        # Ensure end_index is present to form a valid segment
        if support.segment.end_index is None:
            continue  # Skip if end_index is missing, as it's crucial

        # Add 1 to end_index to make it an exclusive end for slicing/range purposes
        # (assuming the API provides an inclusive end_index)
        citation["start_index"] = start_index
        citation["end_index"] = support.segment.end_index

        citation["segments"] = []
        if (
            hasattr(support, "grounding_chunk_indices")
            and support.grounding_chunk_indices
        ):
            for ind in support.grounding_chunk_indices:
                try:
                    chunk = candidate.grounding_metadata.grounding_chunks[ind]
                    resolved_url = resolved_urls_map.get(chunk.web.uri, None)
                    citation["segments"].append(
                        {
                            "label": chunk.web.title.split(".")[:-1][0],
                            "short_url": resolved_url,
                            "value": chunk.web.uri,
                        }
                    )
                except (IndexError, AttributeError, NameError):
                    # Handle cases where chunk, web, uri, or resolved_map might be problematic
                    # For simplicity, we'll just skip adding this particular segment link
                    # In a production system, you might want to log this.
                    pass
        citations.append(citation)
    return citations

In [4]:
# @title
# state.py

from __future__ import annotations

from dataclasses import dataclass, field
from typing import TypedDict

from langgraph.graph import add_messages
from typing_extensions import Annotated


import operator
from dataclasses import dataclass, field
from typing_extensions import Annotated


class OverallState(TypedDict):
    messages: Annotated[list, add_messages]
    search_query: Annotated[list, operator.add]
    web_research_result: Annotated[list, operator.add]
    sources_gathered: Annotated[list, operator.add]
    initial_search_query_count: int
    max_research_loops: int
    research_loop_count: int
    reasoning_model: str


class ReflectionState(TypedDict):
    is_sufficient: bool
    knowledge_gap: str
    follow_up_queries: Annotated[list, operator.add]
    research_loop_count: int
    number_of_ran_queries: int


class Query(TypedDict):
    query: str
    rationale: str


class QueryGenerationState(TypedDict):
    query_list: list[Query]


class WebSearchState(TypedDict):
    search_query: str
    id: str


@dataclass(kw_only=True)
class SearchStateOutput:
    running_summary: str = field(default=None)  # Final report

In [5]:
# @title
# prompts.py
from datetime import datetime


# Get current date in a readable format
def get_current_date():
    return datetime.now().strftime("%B %d, %Y")


query_writer_instructions = """Your goal is to generate sophisticated and diverse web search queries. These queries are intended for an advanced automated web research tool capable of analyzing complex results, following links, and synthesizing information.

Instructions:
- Always prefer a single search query, only add another query if the original question requests multiple aspects or elements and one query is not enough.
- Each query should focus on one specific aspect of the original question.
- Don't produce more than {number_queries} queries.
- Queries should be diverse, if the topic is broad, generate more than 1 query.
- Don't generate multiple similar queries, 1 is enough.
- Query should ensure that the most current information is gathered. The current date is {current_date}.

Format:
- Format your response as a JSON object with ALL three of these exact keys:
   - "rationale": Brief explanation of why these queries are relevant
   - "query": A list of search queries

Example:

Topic: What revenue grew more last year apple stock or the number of people buying an iphone
```json
{{
    "rationale": "To answer this comparative growth question accurately, we need specific data points on Apple's stock performance and iPhone sales metrics. These queries target the precise financial information needed: company revenue trends, product-specific unit sales figures, and stock price movement over the same fiscal period for direct comparison.",
    "query": ["Apple total revenue growth fiscal year 2024", "iPhone unit sales growth fiscal year 2024", "Apple stock price growth fiscal year 2024"],
}}
```

Context: {research_topic}"""


web_searcher_instructions = """Conduct targeted Google Searches to gather the most recent, credible information on "{research_topic}" and synthesize it into a verifiable text artifact.

Instructions:
- Query should ensure that the most current information is gathered. The current date is {current_date}.
- Conduct multiple, diverse searches to gather comprehensive information.
- Consolidate key findings while meticulously tracking the source(s) for each specific piece of information.
- The output should be a well-written summary or report based on your search findings.
- Only include the information found in the search results, don't make up any information.

Research Topic:
{research_topic}
"""

reflection_instructions = """You are an expert research assistant analyzing summaries about "{research_topic}".

Instructions:
- Identify knowledge gaps or areas that need deeper exploration and generate a follow-up query. (1 or multiple).
- If provided summaries are sufficient to answer the user's question, don't generate a follow-up query.
- If there is a knowledge gap, generate a follow-up query that would help expand your understanding.
- Focus on technical details, implementation specifics, or emerging trends that weren't fully covered.

Requirements:
- Ensure the follow-up query is self-contained and includes necessary context for web search.

Output Format:
- Format your response as a JSON object with these exact keys:
   - "is_sufficient": true or false
   - "knowledge_gap": Describe what information is missing or needs clarification
   - "follow_up_queries": Write a specific question to address this gap

Example:
```json
{{
    "is_sufficient": true, // or false
    "knowledge_gap": "The summary lacks information about performance metrics and benchmarks", // "" if is_sufficient is true
    "follow_up_queries": ["What are typical performance benchmarks and metrics used to evaluate [specific technology]?"] // [] if is_sufficient is true
}}
```

Reflect carefully on the Summaries to identify knowledge gaps and produce a follow-up query. Then, produce your output following this JSON format:

Summaries:
{summaries}
"""

answer_instructions = """Generate a high-quality answer to the user's question based on the provided summaries.

Instructions:
- The current date is {current_date}.
- You are the final step of a multi-step research process, don't mention that you are the final step.
- You have access to all the information gathered from the previous steps.
- You have access to the user's question.
- Generate a high-quality answer to the user's question based on the provided summaries and the user's question.
- you MUST include all the citations from the summaries in the answer correctly.

User Context:
- {research_topic}

Summaries:
{summaries}"""

In [6]:
# @title
# tools_and_schemas.py

from typing import List
from pydantic import BaseModel, Field


class SearchQueryList(BaseModel):
    query: List[str] = Field(
        description="A list of search queries to be used for web research."
    )
    rationale: str = Field(
        description="A brief explanation of why these queries are relevant to the research topic."
    )


class Reflection(BaseModel):
    is_sufficient: bool = Field(
        description="Whether the provided summaries are sufficient to answer the user's question."
    )
    knowledge_gap: str = Field(
        description="A description of what information is missing or needs clarification."
    )
    follow_up_queries: List[str] = Field(
        description="A list of follow-up queries to address the knowledge gap."
    )

In [7]:
# @title
# graph.py

import os

from langchain_core.messages import AIMessage
from langgraph.types import Send
from langgraph.graph import StateGraph
from langgraph.graph import START, END
from langchain_core.runnables import RunnableConfig
from google.genai import Client


from langchain_google_genai import ChatGoogleGenerativeAI

from google.colab import userdata

if userdata.get("GOOGLE_API_KEY") is None:
    raise ValueError("GOOGLE_API_KEY is not set")

# Used for Google Search API
genai_client = Client(api_key=userdata.get("GOOGLE_API_KEY"))

# Nodes
def generate_query(state: OverallState, config: RunnableConfig) -> QueryGenerationState:
    """LangGraph node that generates a search queries based on the User's question.

    Uses Gemini 2.0 Flash to create an optimized search query for web research based on
    the User's question.

    Args:
        state: Current graph state containing the User's question
        config: Configuration for the runnable, including LLM provider settings

    Returns:
        Dictionary with state update, including search_query key containing the generated query
    """
    configurable = Configuration.from_runnable_config(config)

    # check for custom initial search query count
    if state.get("initial_search_query_count") is None:
        state["initial_search_query_count"] = configurable.number_of_initial_queries

    # init Gemini 2.0 Flash
    llm = ChatGoogleGenerativeAI(
        model=configurable.query_generator_model,
        temperature=1.0,
        max_retries=2,
        api_key=userdata.get("GOOGLE_API_KEY"),            # os.getenv("GEMINI_API_KEY"),
    )
    structured_llm = llm.with_structured_output(SearchQueryList)

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = query_writer_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        number_queries=state["initial_search_query_count"],
    )
    # Generate the search queries
    result = structured_llm.invoke(formatted_prompt)
    return {"query_list": result.query}


def continue_to_web_research(state: QueryGenerationState):
    """LangGraph node that sends the search queries to the web research node.

    This is used to spawn n number of web research nodes, one for each search query.
    """
    return [
        Send("web_research", {"search_query": search_query, "id": int(idx)})
        for idx, search_query in enumerate(state["query_list"])
    ]


def web_research(state: WebSearchState, config: RunnableConfig) -> OverallState:
    """LangGraph node that performs web research using the native Google Search API tool.

    Executes a web search using the native Google Search API tool in combination with Gemini 2.0 Flash.

    Args:
        state: Current graph state containing the search query and research loop count
        config: Configuration for the runnable, including search API settings

    Returns:
        Dictionary with state update, including sources_gathered, research_loop_count, and web_research_results
    """
    # Configure
    configurable = Configuration.from_runnable_config(config)
    formatted_prompt = web_searcher_instructions.format(
        current_date=get_current_date(),
        research_topic=state["search_query"],
    )

    # Uses the google genai client as the langchain client doesn't return grounding metadata
    response = genai_client.models.generate_content(
        model=configurable.query_generator_model,
        contents=formatted_prompt,
        config={
            "tools": [{"google_search": {}}],
            "temperature": 0,
        },
    )
    # resolve the urls to short urls for saving tokens and time
    resolved_urls = resolve_urls(
        response.candidates[0].grounding_metadata.grounding_chunks, state["id"]
    )
    # Gets the citations and adds them to the generated text
    citations = get_citations(response, resolved_urls)
    modified_text = insert_citation_markers(response.text, citations)
    sources_gathered = [item for citation in citations for item in citation["segments"]]

    return {
        "sources_gathered": sources_gathered,
        "search_query": [state["search_query"]],
        "web_research_result": [modified_text],
    }


def reflection(state: OverallState, config: RunnableConfig) -> ReflectionState:
    """LangGraph node that identifies knowledge gaps and generates potential follow-up queries.

    Analyzes the current summary to identify areas for further research and generates
    potential follow-up queries. Uses structured output to extract
    the follow-up query in JSON format.

    Args:
        state: Current graph state containing the running summary and research topic
        config: Configuration for the runnable, including LLM provider settings

    Returns:
        Dictionary with state update, including search_query key containing the generated follow-up query
    """
    configurable = Configuration.from_runnable_config(config)
    # Increment the research loop count and get the reasoning model
    state["research_loop_count"] = state.get("research_loop_count", 0) + 1
    # reasoning_model = state.get("reasoning_model") or configurable.reasoning_model
    # FIX: Use reflection_model from configuration as a fallback
    reasoning_model = state.get("reasoning_model") or configurable.reflection_model

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = reflection_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        summaries="\n\n---\n\n".join(state["web_research_result"]),
    )
    # init Reasoning Model
    llm = ChatGoogleGenerativeAI(
        model=reasoning_model,
        temperature=1.0,
        max_retries=2,
        api_key=userdata.get("GOOGLE_API_KEY")               #os.getenv("GEMINI_API_KEY"),
    )
    result = llm.with_structured_output(Reflection).invoke(formatted_prompt)

    return {
        "is_sufficient": result.is_sufficient,
        "knowledge_gap": result.knowledge_gap,
        "follow_up_queries": result.follow_up_queries,
        "research_loop_count": state["research_loop_count"],
        "number_of_ran_queries": len(state["search_query"]),
    }


def evaluate_research(
    state: ReflectionState,
    config: RunnableConfig,
) -> OverallState:
    """LangGraph routing function that determines the next step in the research flow.

    Controls the research loop by deciding whether to continue gathering information
    or to finalize the summary based on the configured maximum number of research loops.

    Args:
        state: Current graph state containing the research loop count
        config: Configuration for the runnable, including max_research_loops setting

    Returns:
        String literal indicating the next node to visit ("web_research" or "finalize_summary")
    """
    configurable = Configuration.from_runnable_config(config)
    max_research_loops = (
        state.get("max_research_loops")
        if state.get("max_research_loops") is not None
        else configurable.max_research_loops
    )
    if state["is_sufficient"] or state["research_loop_count"] >= max_research_loops:
        return "finalize_answer"
    else:
        return [
            Send(
                "web_research",
                {
                    "search_query": follow_up_query,
                    "id": state["number_of_ran_queries"] + int(idx),
                },
            )
            for idx, follow_up_query in enumerate(state["follow_up_queries"])
        ]


def finalize_answer(state: OverallState, config: RunnableConfig):
    """LangGraph node that finalizes the research summary.

    Prepares the final output by deduplicating and formatting sources, then
    combining them with the running summary to create a well-structured
    research report with proper citations.

    Args:
        state: Current graph state containing the running summary and sources gathered

    Returns:
        Dictionary with state update, including running_summary key containing the formatted final summary with sources
    """
    configurable = Configuration.from_runnable_config(config)
    # reasoning_model = state.get("reasoning_model") or configurable.reasoning_model
    # FIX: Use answer_model from configuration as a fallback
    reasoning_model = state.get("reasoning_model") or configurable.answer_model

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = answer_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        summaries="\n---\n\n".join(state["web_research_result"]),
    )

    # init Reasoning Model, default to Gemini 2.5 Flash
    llm = ChatGoogleGenerativeAI(
        model=reasoning_model,
        temperature=0,
        max_retries=2,
        api_key=userdata.get("GOOGLE_API_KEY"),               # os.getenv("GEMINI_API_KEY"),
    )
    result = llm.invoke(formatted_prompt)

    # Replace the short urls with the original urls and add all used urls to the sources_gathered
    unique_sources = []
    for source in state["sources_gathered"]:
        if source["short_url"] in result.content:
            result.content = result.content.replace(
                source["short_url"], source["value"]
            )
            unique_sources.append(source)

    return {
        "messages": [AIMessage(content=result.content)],
        "sources_gathered": unique_sources,
    }


# Create our Agent Graph
builder = StateGraph(OverallState, config_schema=Configuration)

# Define the nodes we will cycle between
builder.add_node("generate_query", generate_query)
builder.add_node("web_research", web_research)
builder.add_node("reflection", reflection)
builder.add_node("finalize_answer", finalize_answer)

# Set the entrypoint as `generate_query`
# This means that this node is the first one called
builder.add_edge(START, "generate_query")
# Add conditional edge to continue with search queries in a parallel branch
builder.add_conditional_edges(
    "generate_query", continue_to_web_research, ["web_research"]
)
# Reflect on the web research
builder.add_edge("web_research", "reflection")
# Evaluate the research
builder.add_conditional_edges(
    "reflection", evaluate_research, ["web_research", "finalize_answer"]
)
# Finalize the answer
builder.add_edge("finalize_answer", END)

graph = builder.compile(name="pro-search-agent")

# Enter your prompt below

In [8]:
prompt1 = "Explain in detail on usecases that can leverage LangGraph with Google Gemini models."

In [9]:
state = graph.invoke({"messages": [{"role": "user", "content": prompt1}], "max_research_loops": 3, "initial_search_query_count": 3})

In [10]:
from IPython.display import Markdown

Markdown(state["messages"][-1].content)

LangGraph, an extension of LangChain, provides a robust framework for developing stateful, multi-actor applications powered by Large Language Models (LLMs), including Google's Gemini models [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGstQdA0G2NyxK6qUIW_vpur4t1Hgws_-BBGtChcA31WONzZsUmD1HqLVLRLGVZ3FlfCXofkY_xZjvCItpgpMJRqxCujbqUqvWWNAQ5qnJlPuKYCinAK2N5TDbEwWRN45t0WFY47ISa94DIltgD3-iJFZxGK-ajW_QEZAsoEWP3qrYHRSS3Dwl7-SOxDXM-WAuTMcz6Qg==) [huggingface](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHvmt16SMtPU94AGZM_itbIt0Tr9h1xwoyIGpQC5D2zWuU54W02gicL4wcKcvnDYsln0wi8tz-LS_YzZ0x9FwQPeWoeQIPslIcfZ_rx5hdRXbaJESjyOl4RBIwHQutMcQp-frlBv6Bhv7qJc72dkxVds2yCzG8w_Qk8EsPl8L_Q8RE=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHJNK6udwSk_6juzloBhem_FucJpnHusQwZmqJ22mlAXNoJib4KdGXaF7T6KN-etYm1xve1TRwDX1klAl-OkmVYdeozJLy6a_C5I3DxMZ4hd8GYekjgDtHsI84c1pWSvAhxp7YWUFuhbraQIHE4H40E) [1, 5]. It models workflows as graphs, where each node represents a step (e.g., an LLM call or tool execution) and edges define the flow of control [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGstQdA0G2NyxK6qUIW_vpur4t1Hgws_-BBGtChcA31WONzZsUmD1HqLVLRLGVZ3FlfCXofkY_xZjvCItpgpMJRqxCujbqUqvWWNAQ5qnJlPuKYCinAK2N5TDbEwWRN45t0WFY47ISa94DIltgD3-iJFZxGK-ajW_QEZAsoEWP3qrYHRSS3Dwl7-SOxDXM-WAuTMcz6Qg==) [1, 5]. This graph-based approach is particularly beneficial for complex, stateful workflows that require clear visibility and control over an agent's reasoning process [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGstQdA0G2NyxK6qUIW_vpur4t1Hgws_-BBGtChcA31WONzZsUmD1HqLVLRLGVZ3FlfCXofkY_xZjvCItpgpMJRqxCujbqUqvWWNAQ5qnJlPuKYCinAK2N5TDbEwWRN45t0WFY47ISa94DIltgD3-iJFZxGK-ajW_QEZAsoEWP3qrYHRSS3Dwl7-SOxDXM-WAuTMcz6Qg==) [1].

Here are detailed use cases that can leverage LangGraph with Google Gemini models:

### Core Concepts and Architectural Patterns

LangGraph's design facilitates the creation of sophisticated AI applications through its fundamental components:
*   **State:** A shared data structure (often a TypedDict or Pydantic BaseModel) that maintains the application's current snapshot, persisting across different nodes in a workflow [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGp1wwnv1OYMUuBXqO7VKXvq3dGMf0knV6Qhl8apFr1y6sTCkwmlA9YereGAQvr6cfd5DkSqePZJZCJXnoNECu1I0e2LsozuOxNPw8f0nxS_PVLeK3u08M_zyHh-sYJsSqWG38WWIEWRsJp6VV0JawH_oQRJJo=) [1, 4]. The `AgentState` dataclass, for instance, tracks fields like the user's query, retrieved context, analysis, generated response, and recommended next action [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpv5J4jgfQ2fp5z59nugTSmEQDrRkEyJ6F07UAqmx6r_FwERzTNWb_FAmeyzrntLYgzAXy3nD9npmGvJs4SFgZ54VdOtTzXn1STp2UQtUWzOVfKeNYeHOKrTlBEbr2HyksJuEx2_YXF9ZWLEvjZvmXpk9G2tAZDgtQ2A_f78QZoVeBEOSg5clwMqMXvKlXaDeV5_M5764pYwhSOcgxWL-HYnovn3SFkv7V846Lh0O6Gw3xXPnI1uRfSOtYbOPxE4RVtMCeb18=) [3].
*   **Nodes:** These encapsulate the logic of individual agents or steps within the workflow. Nodes receive the current State as input, perform computations (such as LLM calls or tool invocations), and return an updated State [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGp1wwnv1OYMUuBXqO7VKXvq3dGMf0knV6Qhl8apFr1y6sTCkwmlA9YereGAQvr6cfd5DkSqePZJZCJXnoNECu1I0e2LsozuOxNPw8f0nxS_PVLeK3u08M_zyHh-sYJsSqWG38WWIEWRsJp6VV0JawH_oQRJJo=) [1, 4].
*   **Edges:** Edges dictate the flow of control, defining which node executes next based on the current State. This allows for both fixed transitions and dynamic, conditional logic [1, 2, 4].

These components enable several powerful architectural patterns:

*   **AI Agents:** LangGraph, combined with Gemini models, forms a strong foundation for building AI agents capable of perceiving their environment, making decisions, and executing actions to achieve specific goals [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGstQdA0G2NyxK6qUIW_vpur4t1Hgws_-BBGtChcA31WONzZsUmD1HqLVLRLGVZ3FlfCXofkY_xZjvCItpgpMJRqxCujbqUqvWWNAQ5qnJlPuKYCinAK2N5TDbEwWRN45t0WFY47ISa94DIltgD3-iJFZxGK-ajW_QEZAsoEWP3qrYHRSS3Dwl7-SOxDXM-WAuTMcz6Qg==) [ubos](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGx1RAAvUKSuVJjH9gfvFtjmKgvnKWlhGiaXXyqDvt6NJZoNo47sjt7HolaUg9IBbBF-sB0fycf9vz2s4tTw6k7Vgy6D_Q6jFUewNO8MT88l-1VgBiY5JAgcI-uE36xmq0pU3n-Og2MEXjir-l1TdVI1sgMHiIeTsriwQBD0dma--Hk4psXR5y5rYui2YOlEQzTqmumGdDo8YKz) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHOa4cCbkDSMCEZpMRXQ15l0f_zal1Cz7G2WC-GZYKXXk5XQw0KsHooVpYpMFeTlccvQ_yrCMmORnWW2Qu1U3ftX3VBbdA6eafpGWfsC_M3KULUR5Bc8iD90k1wC5uKHgszYq1pONz1CdT-IhvmREx68FvnpAQ=) [1].
*   **Stateful Workflows:** LangGraph and Gemini models structure AI reasoning into stateful workflows, allowing an incoming query to progress through a series of nodes for tasks like routing, analysis, research, response generation, and validation [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpv5J4jgfQ2fp5z59nugTSmEQDrRkEyJ6F07UAqmx6r_FwERzTNWb_FAmeyzrntLYgzAXy3nD9npmGvJs4SFgZ54VdOtTzXn1STp2UQtUWzOVfKeNYeHOKrTlBEbr2HyksJuEx2_YXF9ZWLEvjZvmXpk9G2tAZDgtQ2A_f78QZoVeBEOSg5clwMqMXvKlXaDeV5_M5764pYwhSOcgxWL-HYnovn3SFkv7V846Lh0O6Gw3xXPnI1uRfSOtYbOPxE4RVtMCeb18=) [ubos](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGx1RAAvUKSuVJjH9gfvFtjmKgvnKWlhGiaXXyqDvt6NJZoNo47sjt7HolaUg9IBbBF-sB0fycf9vz2s4tTw6k7Vgy6D_Q6jFUewNO8MT88l-1VgBiY5JAgcI-uE36xmq0pU3n-Og2MEXjir-l1TdVI1sgMHiIeTsriwQBD0dma--Hk4psXR5y5rYui2YOlEQzTqmumGdDo8YKz) [2, 3]. The `GraphAIAgent` class, for example, orchestrates nodes such as router, analyzer, researcher, responder, and validator [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpv5J4jgfQ2fp5z59nugTSmEQDrRkEyJ6F07UAqmx6r_FwERzTNWb_FAmeyzrntLYgzAXy3nD9npmGvJs4SFgZ54VdOtTzXn1STp2UQtUWzOVfKeNYeHOKrTlBEbr2HyksJuEx2_YXF9ZWLEvjZvmXpk9G2tAZDgtQ2A_f78QZoVeBEOSg5clwMqMXvKlXaDeV5_M5764pYwhSOcgxWL-HYnovn3SFkv7V846Lh0O6Gw3xXPnI1uRfSOtYbOPxE4RVtMCeb18=) [ubos](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGx1RAAvUKSuVJjH9gfvFtjmKgvnKWlhGiaXXyqDvt6NJZoNo47sjt7HolaUg9IBbBF-sB0fycf9vz2s4tTw6k7Vgy6D_Q6jFUewNO8MT88l-1VgBiY5JAgcI-uE36xmq0pU3n-Og2MEXjir-l1TdVI1sgMHiIeTsriwQBD0dma--Hk4psXR5y5rYui2YOlEQzTqmumGdDo8YKz) [2].
*   **ReAct Agents:** LangGraph is particularly well-suited for constructing ReAct (Reasoning and Acting) Agents. These agents combine LLM reasoning with action execution, iteratively thinking, using tools, and acting on observations to dynamically adapt and achieve user goals [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGk3U4LFwGzRE509Ya98ghoYuI7ooo5planG5atbjq6P62m0swaoH1mspiuBkTOnCo3p3F09OtvOr8TBtjcoK_fTskXnk1ccKtpDDeU74yjvCiNz-WzFL2VWHl9Bk6K7dIFZxaPhYFThyzgCxeP5yjsr6I=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHOa4cCbkDSMCEZpMRXQ15l0f_zal1Cz7G2WC-GZYKXXk5XQw0KsHooVpYpMFeTlccvQ_yrCMmORnWW2Qu1U3ftX3VBbdA6eafpGWfsC_M3KULUR5Bc8iD90k1wC5uKHgszYq1pONz1CdT-IhvmREx68FvnpAQ=) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [huggingface](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH69nCSvxyR6rjSd4enlgbupDmKMNBZ4tFFf_WDNomOdWCxP32s1BltCF_8U4maHtyfoJcL2tSPl5gdaNMulHv9-DQR55Pz1Zuvfa8EDlHzJp27VZCmNwQet1LPRLKkcGmuAGg_fzShLqonlmnhZGcXfDNK-uHDOoF6ajDyBE0YKYU=) [4, 9].
*   **Multi-Agent Systems:** LangGraph simplifies the development of multi-agent systems by enabling the definition of agents with specific roles and goals, and then assigning tasks to them [1]. Several architectures can be employed for connecting agents:
    *   **Network:** Each agent can communicate with every other agent, and any agent can decide which other agent to call next [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF0QPL8ULBPwdLoKyg60Dna8Fyw0_R_7_flSZB5FjLxU9HgKG3aY4ICe3C7dkDOAkmcOpTbkhZgWVGXDrKw9rt-qO-g4KkPiaMWSx1CR28Q9ajZajQ9O3vzT6kHxjjJUz9DThf08G3V2FS-D9iJRh299i3VJO0Au7tc) [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE5HgrK8nj4KMKRsSsbJNmVwIXC04cAa6fqN5aeeCjKx5lHvoV-N2YarqDv9EymXK0uv2iWZu2jaN3dyGbGMfKiKfmnDVK_EPeqaG37ZsgR_tOnpxITbEDkXb5uzl8dUFvLAX9McFMATQ4Am3sakmPPdYGVGL4KII6Yl3LQbSARwpccia1KjegvPLrCTdrqC4djGHPHZwIgeVUU0QpG) [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEx-Ke30g0yCSKbfGD233KQpBDliiSepkXX8xqBlOOJC90Yi4TWvhN3idVeCswgI728UuLicjBymTYSJcNg-4lipvtsxU7vCi74STU02MORTo4LXU_IP9CZTdY9lHsSE6Hrvh8G36saYQRrdX7jxicv7I18JKF8tETxdrAaSmENqfxKNcDT5qGxMLS2sF5O42911JD2c3WYer1mGh-fTYzpJQ1LXWGduuc1lnVhZTmQCcxmgQbx31ZWS0ZO7Rm87QcwUgiDDgTcnofTitcXCCvTHgr-c4KGf_rPuJ8rEndL_Q==) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF0QPL8ULBPwdLoKyg60Dna8Fyw0_R_7_flSZB5FjLxU9HgKG3aY4ICe3C7dkDOAkmcOpTbkhZgWVGXDrKw9rt-qO-g4KkPiaMWSx1CR28Q9ajZajQ9O3vzT6kHxjjJUz9DThf08G3V2FS-D9iJRh299i3VJO0Au7tc) [langchain](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGhewNsTDggem59eN_--oak0eZgE7ebTqXpl0grM-jt-xSsO0HNxMNPNG3ilcmZWnFBq_h_PQw60D8l10awO7z4K4_tlK36LXP9-ZuN2xsjByoxFza3VdKC6b0SZ1ds8rPBkhZo2CUR19I=) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGlLcJk-j2I5AbMnP4SNQqic6NYMCJKi_5FRkWByuz8_oemFLtc4V_dHwRePkGEzPAkkI4RunZ6VUk5024qiWZpZT_DHoBxcYCpigAz3qP4Ou_Lgigcj0Ufo-MV93fLGn8coACIPYVtr7tNP4f6cWQ6fvRhfZbV73slWo_vcTRxmM3iqvbaU3HiDz1nM6FLcOVSNrfaIFv9cNhqXDEPrCF2jAjKz95mi2syHolVUrEXZxs4) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzeCK4r-cyJe4yJGzU4L-nHGcLZbfv8cd3zsMg0BlTMxmamjVuW-9Uam0RwjArvnj41irG22w0_3O1EXzipDC95fw9LaegR92IRMVg5RxTY-PgzsVpEfg7CI2JGRTbmsqJZlU637hPL5dFoAo3J69OnMERjuCiaKjtvMi9YtdSnwE=) [analyticsvidhya](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=0) [9].
    *   **Supervisor:** Each agent communicates with a single supervisor agent, which makes decisions on which agent should be called next [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF0QPL8ULBPwdLoKyg60Dna8Fyw0_R_7_flSZB5FjLxU9HgKG3aY4ICe3C7dkDOAkmcOpTbkhZgWVGXDrKw9rt-qO-g4KkPiaMWSx1CR28Q9ajZajQ9O3vzT6kHxjjJUz9DThf08G3V2FS-D9iJRh299i3VJO0Au7tc) [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE5HgrK8nj4KMKRsSsbJNmVwIXC04cAa6fqN5aeeCjKx5lHvoV-N2YarqDv9EymXK0uv2iWZu2jaN3dyGbGMfKiKfmnDVK_EPeqaG37ZsgR_tOnpxITbEDkXb5uzl8dUFvLAX9McFMATQ4Am3sakmPPdYGVGL4KII6Yl3LQbSARwpccia1KjegvPLrCTdrqC4djGHPHZwIgeVUU0QpG) [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEx-Ke30g0yCSKbfGD233KQpBDliiSepkXX8xqBlOOJC90Yi4TWvhN3idVeCswgI728UuLicjBymTYSJcNg-4lipvtsxU7vCi74STU02MORTo4LXU_IP9CZTdY9lHsSE6Hrvh8G36saYQRrdX7jxicv7I18JKF8tETxdrAaSmENqfxKNcDT5qGxMLS2sF5O42911JD2c3WYer1mGh-fTYzpJQ1LXWGduuc1lnVhZTmQCcxmgQbx31ZWS0ZO7Rm87QcwUgiDDgTcnofTitcXCCvTHgr-c4KGf_rPuJ8rEndL_Q==) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF0QPL8ULBPwdLoKyg60Dna8Fyw0_R_7_flSZB5FjLxU9HgKG3aY4ICe3C7dkDOAkmcOpTbkhZgWVGXDrKw9rt-qO-g4KkPiaMWSx1CR28Q9ajZajQ9O3vzT6kHxjjJUz9DThf08G3V2FS-D9iJRh299i3VJO0Au7tc) [langchain](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGhewNsTDggem59eN_--oak0eZgE7ebTqXpl0grM-jt-xSsO0HNxMNPNG3ilcmZWnFBq_h_PQw60D8l10awO7z4K4_tlK36LXP9-ZuN2xsjByoxFza3VdKC6b0SZ1ds8rPBkhZo2CUR19I=) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGlLcJk-j2I5AbMnP4SNQqic6NYMCJKi_5FRkWByuz8_oemFLtc4V_dHwRePkGEzPAkkI4RunZ6VUk5024qiWZpZT_DHoBxcYCpigAz3qP4Ou_Lgigcj0Ufo-MV93fLGn8coACIPYVtr7tNP4f6cWQ6fvRhfZbV73slWo_vcTRxmM3iqvbaU3HiDz1nM6FLcOVSNrfaIFv9cNhqXDEPrCF2jAjKz95mi2syHolVUrEXZxs4) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzeCK4r-cyJe4yJGzU4L-nHGcLZbfv8cd3zsMg0BlTMxmamjVuW-9Uam0RwjArvnj41irG22w0_3O1EXzipDC95fw9LaegR92IRMVg5RxTY-PgzsVpEfg7CI2JGRTbmsqJZlU637hPL5dFoAo3J69OnMERjuCiaKjtvMi9YtdSnwE=) [analyticsvidhya](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=0) [9].
    *   **Supervisor (tool-calling):** A specialized case where individual agents are represented as tools, and a supervisor agent uses a tool-calling LLM to decide which agent tool to invoke [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF0QPL8ULBPwdLoKyg60Dna8Fyw0_R_7_flSZB5FjLxU9HgKG3aY4ICe3C7dkDOAkmcOpTbkhZgWVGXDrKw9rt-qO-g4KkPiaMWSx1CR28Q9ajZajQ9O3vzT6kHxjjJUz9DThf08G3V2FS-D9iJRh299i3VJO0Au7tc) [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE5HgrK8nj4KMKRsSsbJNmVwIXC04cAa6fqN5aeeCjKx5lHvoV-N2YarqDv9EymXK0uv2iWZu2jaN3dyGbGMfKiKfmnDVK_EPeqaG37ZsgR_tOnpxITbEDkXb5uzl8dUFvLAX9McFMATQ4Am3sakmPPdYGVGL4KII6Yl3LQbSARwpccia1KjegvPLrCTdrqC4djGHPHZwIgeVUU0QpG) [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEx-Ke30g0yCSKbfGD233KQpBDliiSepkXX8xqBlOOJC90Yi4TWvhN3idVeCswgI728UuLicjBymTYSJcNg-4lipvtsxU7vCi74STU02MORTo4LXU_IP9CZTdY9lHsSE6Hrvh8G36saYQRrdX7jxicv7I18JKF8tETxdrAaSmENqfxKNcDT5qGxMLS2sF5O42911JD2c3WYer1mGh-fTYzpJQ1LXWGduuc1lnVhZTmQCcxmgQbx31ZWS0ZO7Rm87QcwUgiDDgTcnofTitcXCCvTHgr-c4KGf_rPuJ8rEndL_Q==) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF0QPL8ULBPwdLoKyg60Dna8Fyw0_R_7_flSZB5FjLxU9HgKG3aY4ICe3C7dkDOAkmcOpTbkhZgWVGXDrKw9rt-qO-g4KkPiaMWSx1CR28Q9ajZajQ9O3vzT6kHxjjJUz9DThf08G3V2FS-D9iJRh299i3VJO0Au7tc) [langchain](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGhewNsTDggem59eN_--oak0eZgE7ebTqXpl0grM-jt-xSsO0HNxMNPNG3ilcmZWnFBq_h_PQw60D8l10awO7z4K4_tlK36LXP9-ZuN2xsjByoxFza3VdKC6b0SZ1ds8rPBkhZo2CUR19I=) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGlLcJk-j2I5AbMnP4SNQqic6NYMCJKi_5FRkWByuz8_oemFLtc4V_dHwRePkGEzPAkkI4RunZ6VUk5024qiWZpZT_DHoBxcYCpigAz3qP4Ou_Lgigcj0Ufo-MV93fLGn8coACIPYVtr7tNP4f6cWQ6fvRhfZbV73slWo_vcTRxmM3iqvbaU3HiDz1nM6FLcOVSNrfaIFv9cNhqXDEPrCF2jAjKz95mi2syHolVUrEXZxs4) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzeCK4r-cyJe4yJGzU4L-nHGcLZbfv8cd3zsMg0BlTMxmamjVuW-9Uam0RwjArvnj41irG22w0_3O1EXzipDC95fw9LaegR92IRMVg5RxTY-PgzsVpEfg7CI2JGRTbmsqJZlU637hPL5dFoAo3J69OnMERjuCiaKjtvMi9YtdSnwE=) [analyticsvidhya](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=0) [9].
    *   **Hierarchical:** A multi-agent system featuring a supervisor of supervisors, enabling more intricate control flows [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF0QPL8ULBPwdLoKyg60Dna8Fyw0_R_7_flSZB5FjLxU9HgKG3aY4ICe3C7dkDOAkmcOpTbkhZgWVGXDrKw9rt-qO-g4KkPiaMWSx1CR28Q9ajZajQ9O3vzT6kHxjjJUz9DThf08G3V2FS-D9iJRh299i3VJO0Au7tc) [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE5HgrK8nj4KMKRsSsbJNmVwIXC04cAa6fqN5aeeCjKx5lHvoV-N2YarqDv9EymXK0uv2iWZu2jaN3dyGbGMfKiKfmnDVK_EPeqaG37ZsgR_tOnpxITbEDkXb5uzl8dUFvLAX9McFMATQ4Am3sakmPPdYGVGL4KII6Yl3LQbSARwpccia1KjegvPLrCTdrqC4djGHPHZwIgeVUU0QpG) [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEx-Ke30g0yCSKbfGD233KQpBDliiSepkXX8xqBlOOJC90Yi4TWvhN3idVeCswgI728UuLicjBymTYSJcNg-4lipvtsxU7vCi74STU02MORTo4LXU_IP9CZTdY9lHsSE6Hrvh8G36saYQRrdX7jxicv7I18JKF8tETxdrAaSmENqfxKNcDT5qGxMLS2sF5O42911JD2c3WYer1mGh-fTYzpJQ1LXWGduuc1lnVhZTmQCcxmgQbx31ZWS0ZO7Rm87QcwUgiDDgTcnofTitcXCCvTHgr-c4KGf_rPuJ8rEndL_Q==) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF0QPL8ULBPwdLoKyg60Dna8Fyw0_R_7_flSZB5FjLxU9HgKG3aY4ICe3C7dkDOAkmcOpTbkhZgWVGXDrKw9rt-qO-g4KkPiaMWSx1CR28Q9ajZajQ9O3vzT6kHxjjJUz9DThf08G3V2FS-D9iJRh299i3VJO0Au7tc) [langchain](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGhewNsTDggem59eN_--oak0eZgE7ebTqXpl0grM-jt-xSsO0HNxMNPNG3ilcmZWnFBq_h_PQw60D8l10awO7z4K4_tlK36LXP9-ZuN2xsjByoxFza3VdKC6b0SZ1ds8rPBkhZo2CUR19I=) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGlLcJk-j2I5AbMnP4SNQqic6NYMCJKi_5FRkWByuz8_oemFLtc4V_dHwRePkGEzPAkkI4RunZ6VUk5024qiWZpZT_DHoBxcYCpigAz3qP4Ou_Lgigcj0Ufo-MV93fLGn8coACIPYVtr7tNP4f6cWQ6fvRhfZbV73slWo_vcTRxmM3iqvbaU3HiDz1nM6FLcOVSNrfaIFv9cNhqXDEPrCF2jAjKz95mi2syHolVUrEXZxs4) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzeCK4r-cyJe4yJGzU4L-nHGcLZbfv8cd3zsMg0BlTMxmamjVuW-9Uam0RwjArvnj41irG22w0_3O1EXzipDC95fw9LaegR92IRMVg5RxTY-PgzsVpEfg7CI2JGRTbmsqJZlU637hPL5dFoAo3J69OnMERjuCiaKjtvMi9YtdSnwE=) [analyticsvidhya](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=0) [9].

### Key Use Cases and Implementation Examples

LangGraph with Google Gemini models can be applied to a wide array of complex scenarios:

*   **Research-Augmented Conversational AI (Deep Research Agent):** This is a prominent use case where LangGraph and Gemini models can build conversational AI that provides answers with a transparent research process and citations [huggingface](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHvmt16SMtPU94AGZM_itbIt0Tr9h1xwoyIGpQC5D2zWuU54W02gicL4wcKcvnDYsln0wi8tz-LS_YzZ0x9FwQPeWoeQIPslIcfZ_rx5hdRXbaJESjyOl4RBIwHQutMcQp-frlBv6Bhv7qJc72dkxVds2yCzG8w_Qk8EsPl8L_Q8RE=) [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHcYcpZdhIDia2dRPbxXFimlM3Rs-bXmZj7O5arNhyD3YJhfGK9Aiy_HDGjfUVm_EWaeI0dVganlVyFb7KhpNNUKWNa8oOL0Wh4Oq39L0R243yUciDn-5WLx2nFvA8gWZq7JJsKsgO1XL-69g2qBadUbjPQBm7cGZIG1FhqUydWYh8=) [5, 6]. An example project, the "Gemini Fullstack LangGraph Quickstart," demonstrates a backend agent that performs in-depth research. This agent dynamically generates search terms, queries the web, evaluates gathered information, and iteratively refines its search strategy to deliver well-supported responses [huggingface](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHvmt16SMtPU94AGZM_itbIt0Tr9h1xwoyIGpQC5D2zWuU54W02gicL4wcKcvnDYsln0wi8tz-LS_YzZ0x9FwQPeWoeQIPslIcfZ_rx5hdRXbaJESjyOl4RBIwHQutMcQp-frlBv6Bhv7qJc72dkxVds2yCzG8w_Qk8EsPl8L_Q8RE=) [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHcYcpZdhIDia2dRPbxXFimlM3Rs-bXmZj7O5arNhyD3YJhfGK9Aiy_HDGjfUVm_EWaeI0dVganlVyFb7KhpNNUKWNa8oOL0Wh4Oq39L0R243yUciDn-5WLx2nFvA8gWZq7JJsKsgO1XL-69g2qBadUbjPQBm7cGZIG1FhqUydWYh8=) [5, 6, 7].
*   **Multimodal Agents:** LangGraph can be combined with Gemini to create multimodal agents capable of detecting objects in images, audio, and video [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGlLcJk-j2I5AbMnP4SNQqic6NYMCJKi_5FRkWByuz8_oemFLtc4V_dHwRePkGEzPAkkI4RunZ6VUk5024qiWZpZT_DHoBxcYCpigAz3qP4Ou_Lgigcj0Ufo-MV93fLGn8coACIPYVtr7tNP4f6cWQ6fvRhfZbV73slWo_vcTRxmM3iqvbaU3HiDz1nM6FLcOVSNrfaIFv9cNhqXDEPrCF2jAjKz95mi2syHolVUrEXZxs4) [8]. This is valuable for applications like content moderation, multimedia search, and retrieval [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGlLcJk-j2I5AbMnP4SNQqic6NYMCJKi_5FRkWByuz8_oemFLtc4V_dHwRePkGEzPAkkI4RunZ6VUk5024qiWZpZT_DHoBxcYCpigAz3qP4Ou_Lgigcj0Ufo-MV93fLGn8coACIPYVtr7tNP4f6cWQ6fvRhfZbV73slWo_vcTRxmM3iqvbaU3HiDz1nM6FLcOVSNrfaIFv9cNhqXDEPrCF2jAjKz95mi2syHolVUrEXZxs4) [6]. Such systems often involve an Orchestrator Agent that dispatches tasks to specialized worker agents (e.g., `image_agent`, `audio_agent`, `video_agent`) for analyzing different file types [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE5HgrK8nj4KMKRsSsbJNmVwIXC04cAa6fqN5aeeCjKx5lHvoV-N2YarqDv9EymXK0uv2iWZu2jaN3dyGbGMfKiKfmnDVK_EPeqaG37ZsgR_tOnpxITbEDkXb5uzl8dUFvLAX9McFMATQ4Am3sakmPPdYGVGL4KII6Yl3LQbSARwpccia1KjegvPLrCTdrqC4djGHPHZwIgeVUU0QpG) [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEx-Ke30g0yCSKbfGD233KQpBDliiSepkXX8xqBlOOJC90Yi4TWvhN3idVeCswgI728UuLicjBymTYSJcNg-4lipvtsxU7vCi74STU02MORTo4LXU_IP9CZTdY9lHsSE6Hrvh8G36saYQRrdX7jxicv7I18JKF8tETxdrAaSmENqfxKNcDT5qGxMLS2sF5O42911JD2c3WYer1mGh-fTYzpJQ1LXWGduuc1lnVhZTmQCcxmgQbx31ZWS0ZO7Rm87QcwUgiDDgTcnofTitcXCCvTHgr-c4KGf_rPuJ8rEndL_Q==) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF0QPL8ULBPwdLoKyg60Dna8Fyw0_R_7_flSZB5FjLxU9HgKG3aY4ICe3C7dkDOAkmcOpTbkhZgWVGXDrKw9rt-qO-g4KkPiaMWSx1CR28Q9ajZajQ9O3vzT6kHxjjJUz9DThf08G3V2FS-D9iJRh299i3VJO0Au7tc) [langchain](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGhewNsTDggem59eN_--oak0eZgE7ebTqXpl0grM-jt-xSsO0HNxMNPNG3ilcmZWnFBq_h_PQw60D8l10awO7z4K4_tlK36LXP9-ZuN2xsjByoxFza3VdKC6b0SZ1ds8rPBkhZo2CUR19I=) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGlLcJk-j2I5AbMnP4SNQqic6NYMCJKi_5FRkWByuz8_oemFLtc4V_dHwRePkGEzPAkkI4RunZ6VUk5024qiWZpZT_DHoBxcYCpigAz3qP4Ou_Lgigcj0Ufo-MV93fLGn8coACIPYVtr7tNP4f6cWQ6fvRhfZbV73slWo_vcTRxmM3iqvbaU3HiDz1nM6FLcOVSNrfaIFv9cNhqXDEPrCF2jAjKz95mi2syHolVUrEXZxs4) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzeCK4r-cyJe4yJGzU4L-nHGcLZbfv8cd3zsMg0BlTMxmamjVuW-9Uam0RwjArvnj41irG22w0_3O1EXzipDC95fw9LaegR92IRMVg5RxTY-PgzsVpEfg7CI2JGRTbmsqJZlU637hPL5dFoAo3J69OnMERjuCiaKjtvMi9YtdSnwE=) [analyticsvidhya](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=0) [6].
*   **Intelligent Conversational Agents (e.g., Sightseeing Agent):** An intelligent, conversational sightseeing agent can be built using LangGraph, Gemini LLM, and Google Maps APIs. Its core logic is structured as a graph, explicitly defining the flow of information and actions needed to help a user plan a trip. This involves a defined state schema (e.g., `TravelPlanState`) and conditional edges for dynamic routing based on the agent's output [kaggle](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=1) [2].
*   **Specialized Research Agents (e.g., Paper-to-Voice Assistant):** A research agent can be developed to summarize research papers, potentially using a vision model to infer information from documents, and then convert the summary to voice [analyticsvidhya](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=0) [5].
*   **Educational Systems (e.g., Aidemy):** Multi-agent systems on Google Cloud can be created for educational purposes, such as generating on-demand quizzes and dynamic content, leveraging Gemini models within an event-driven architecture [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzeCK4r-cyJe4yJGzU4L-nHGcLZbfv8cd3zsMg0BlTMxmamjVuW-9Uam0RwjArvnj41irG22w0_3O1EXzipDC95fw9LaegR92IRMVg5RxTY-PgzsVpEfg7CI2JGRTbmsqJZlU637hPL5dFoAo3J69OnMERjuCiaKjtvMi9YtdSnwE=) [8].
*   **Modular Enterprise AI Assistants (e.g., Vodafone):** Companies like Vodafone have utilized LangGraph to construct modular AI assistants. These assistants are designed as subgraphs, with each subgraph containing specific tools responsible for a particular task. This modularity allows for easy addition of new capabilities, such as data collection, processing, report generation, and advanced reasoning using Retrieval-Augmented Generation (RAG) pipelines [langchain](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGhewNsTDggem59eN_--oak0eZgE7ebTqXpl0grM-jt-xSsO0HNxMNPNG3ilcmZWnFBq_h_PQw60D8l10awO7z4K4_tlK36LXP9-ZuN2xsjByoxFza3VdKC6b0SZ1ds8rPBkhZo2CUR19I=) [15].
*   **Tool Integration:** Gemini models' function calling capabilities allow LangGraph agents to interact seamlessly with external tools, APIs, and data sources, enabling them to perform real-world actions. This includes integrating tools for web scraping, search, and specialized APIs for domains like finance and maps [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE68XprQniA6e1-BJqh4c-QIPHWDjq1SeyZIFyTb0JWc7IOGol45eMhXytr58Z8IX3lGwYnEImGi_ZEZucDV3rLvUK5A6LC_yiazMGtJMkcwxe6VnQmjamgGLz6pHC6Zyt3WKuZSNeYJiDjDWKUzL58muq-CK7yD3ojGRYc5vJmhvDS8Q-hX538kF01LDW4njeBdJxEtB2Zng2KIJcI4cqrBNOOJ38dgeBx-D3KtqTRXqQh_kkJ9UBYpTjPGXrSPPEO0aC0CbxYQ-YIxluKO12zdyWpZdky-edyeRaqfVFTKg==) [1, 12]. A simple ReAct agent for weather information, for instance, uses a tool to find current weather for a specified location, with the state maintaining conversation history and step count [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [1].

### Practical Implementation Details

When building LangGraph agents with Gemini, several practical aspects are key:

*   **Node Configurations:** Common node types include `call_tool` (often using LangGraph's prebuilt `ToolNode`) for executing tool methods, and `call_model` for invoking the Gemini model [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGp1wwnv1OYMUuBXqO7VKXvq3dGMf0knV6Qhl8apFr1y6sTCkwmlA9YereGAQvr6cfd5DkSqePZJZCJXnoNECu1I0e2LsozuOxNPw8f0nxS_PVLeK3u08M_zyHh-sYJsSqWG38WWIEWRsJp6VV0JawH_oQRJJo=) [1]. Custom nodes can also be added for structured output or self-verification/reflection [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGp1wwnv1OYMUuBXqO7VKXvq3dGMf0knV6Qhl8apFr1y6sTCkwmlA9YereGAQvr6cfd5DkSqePZJZCJXnoNECu1I0e2LsozuOxNPw8f0nxS_PVLeK3u08M_zyHh-sYJsSqWG38WWIEWRsJp6VV0JawH_oQRJJo=) [1, 4]. A `user_input` node can display agent responses and prompt for the next user query [kaggle](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=1) [2].
*   **Edge Configurations:** Edges define transitions. A `should_continue` edge can decide whether to call a tool or the model [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGp1wwnv1OYMUuBXqO7VKXvq3dGMf0knV6Qhl8apFr1y6sTCkwmlA9YereGAQvr6cfd5DkSqePZJZCJXnoNECu1I0e2LsozuOxNPw8f0nxS_PVLeK3u08M_zyHh-sYJsSqWG38WWIEWRsJp6VV0JawH_oQRJJo=) [1, 4]. Conditional edges enable dynamic workflow based on node outcomes [2, 8], while some transitions might be fixed (e.g., from tools back to the agent) [kaggle](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=1) [2].
*   **State Management:** LangGraph provides helpers like `add_messages` for efficiently updating message lists within the agent's state [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [1].
*   **Tool Integration:** Tools are bound to the LLM, allowing the Gemini model to understand their purpose and request their execution with appropriate arguments [kaggle](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=1) [2].
*   **Dynamic Control Flow:** LangGraph empowers LLMs to influence parts of the application's control flow using `Command` objects [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF0QPL8ULBPwdLoKyg60Dna8Fyw0_R_7_flSZB5FjLxU9HgKG3aY4ICe3C7dkDOAkmcOpTbkhZgWVGXDrKw9rt-qO-g4KkPiaMWSx1CR28Q9ajZajQ9O3vzT6kHxjjJUz9DThf08G3V2FS-D9iJRh299i3VJO0Au7tc) [9].
*   **Human-in-the-Loop:** LangGraph supports incorporating human intervention and oversight into agent workflows, which is crucial for complex or sensitive applications [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=2) [11].

### Setting Up and Supporting Frameworks

To set up a LangGraph agent with Gemini models, you typically need to:
1.  Install necessary packages: `pip install langgraph langchain-google-genai python-dotenv` [ubos](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGx1RAAvUKSuVJjH9gfvFtjmKgvnKWlhGiaXXyqDvt6NJZoNo47sjt7HolaUg9IBbBF-sB0fycf9vz2s4tTw6k7Vgy6D_Q6jFUewNO8MT88l-1VgBiY5JAgcI-uE36xmq0pU3n-Og2MEXjir-l1TdVI1sgMHiIeTsriwQBD0dma--Hk4psXR5y5rYui2YOlEQzTqmumGdDo8YKz) [2].
2.  Import essential modules, including `ChatGoogleGenerativeAI` for Gemini interaction and `StateGraph` for state management [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpv5J4jgfQ2fp5z59nugTSmEQDrRkEyJ6F07UAqmx6r_FwERzTNWb_FAmeyzrntLYgzAXy3nD9npmGvJs4SFgZ54VdOtTzXn1STp2UQtUWzOVfKeNYeHOKrTlBEbr2HyksJuEx2_YXF9ZWLEvjZvmXpk9G2tAZDgtQ2A_f78QZoVeBEOSg5clwMqMXvKlXaDeV5_M5764pYwhSOcgxWL-HYnovn3SFkv7V846Lh0O6Gw3xXPnI1uRfSOtYbOPxE4RVtMCeb18=) [3].
3.  Define the agent's state and workflow using a dataclass to track key fields [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpv5J4jgfQ2fp5z59nugTSmEQDrRkEyJ6F07UAqmx6r_FwERzTNWb_FAmeyzrntLYgzAXy3nD9npmGvJs4SFgZ54VdOtTzXn1STp2UQtUWzOVfKeNYeHOKrTlBEbr2HyksJuEx2_YXF9ZWLEvjZvmXpk9G2tAZDgtQ2A_f78QZoVeBEOSg5clwMqMXvKlXaDeV5_M5764pYwhSOcgxWL-HYnovn3SFkv7V846Lh0O6Gw3xXPnI1uRfSOtYbOPxE4RVtMCeb18=) [2, 3].
4.  Configure the Gemini model with desired settings (version, temperature, max output tokens) [10, 11].
5.  Define and instantiate tools the agent can use [10].
6.  Customize the prompt template to guide the agent's behavior and orchestrate the workflow by defining nodes and edges [10].
7.  Access Google AI models, requiring a Google Account and a Google AI API key [langchain](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHfr_T5S_BDF_NO8bMd5fszd3exisQz8j6aH8olxRVtlIYpFEmuJliTujeriAtVxHan-CUmtXherblPnPiTWTZ11diJc26p2P7EZpjMEqwJm_f3Tpo2T8pMrFDkZL68r7m6J5nuZTFcSDH0sFwB49fDbzhEbMC5ZpMigQ4P6tTh8Ikmoto=) [7].

Supporting frameworks and tools include:
*   **LangChain:** Provides utilities to chain together LLM calls and external data sources [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGlLcJk-j2I5AbMnP4SNQqic6NYMCJKi_5FRkWByuz8_oemFLtc4V_dHwRePkGEzPAkkI4RunZ6VUk5024qiWZpZT_DHoBxcYCpigAz3qP4Ou_Lgigcj0Ufo-MV93fLGn8coACIPYVtr7tNP4f6cWQ6fvRhfZbV73slWo_vcTRxmM3iqvbaU3HiDz1nM6FLcOVSNrfaIFv9cNhqXDEPrCF2jAjKz95mi2syHolVUrEXZxs4) [6].
*   **Agent Development Kit (ADK):** A new open-source framework from Google designed to simplify the full-stack, end-to-end development of agents and multi-agent systems [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE5HgrK8nj4KMKRsSsbJNmVwIXC04cAa6fqN5aeeCjKx5lHvoV-N2YarqDv9EymXK0uv2iWZu2jaN3dyGbGMfKiKfmnDVK_EPeqaG37ZsgR_tOnpxITbEDkXb5uzl8dUFvLAX9McFMATQ4Am3sakmPPdYGVGL4KII6Yl3LQbSARwpccia1KjegvPLrCTdrqC4djGHPHZwIgeVUU0QpG) [6, 14].
*   **LiteLLM:** Allows integration with a wide selection of models from various providers beyond Google, such as Anthropic, Meta, Mistral AI, and AI21 Labs [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE5HgrK8nj4KMKRsSsbJNmVwIXC04cAa6fqN5aeeCjKx5lHvoV-N2YarqDv9EymXK0uv2iWZu2jaN3dyGbGMfKiKfmnDVK_EPeqaG37ZsgR_tOnpxITbEDkXb5uzl8dUFvLAX9McFMATQ4Am3sakmPPdYGVGL4KII6Yl3LQbSARwpccia1KjegvPLrCTdrqC4djGHPHZwIgeVUU0QpG) [14].

By leveraging these capabilities, developers can build highly sophisticated and adaptable AI applications with Google Gemini models and LangGraph.

# Enter follow-up prompt

In [11]:
prompt2 = "Expand on usecases for Financial Services"

In [12]:
state = graph.invoke({"messages": state["messages"] + [{"role": "user", "content": prompt2}]})
Markdown(state["messages"][-1].content)

LangGraph, an extension of LangChain, provides a robust framework for developing stateful, multi-actor applications powered by Large Language Models (LLMs), including Google's Gemini models [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGstQdA0G2NyxK6qUIW_vpur4t1Hgws_-BBGtChcA31WONzZsUmD1HqLVLRLGVZ3FlfCXofkY_xZjvCItpgpMJRqxCujbqUqvWWNAQ5qnJlPuKYCinAK2N5TDbEwWRN45t0WFY47ISa94DIltgD3-iJFZxGK-ajW_QEZAsoEWP3qrYHRSS3Dwl7-SOxDXM-WAuTMcz6Qg==) [huggingface](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHvmt16SMtPU94AGZM_itbIt0Tr9h1xwoyIGpQC5D2zWuU54W02gicL4wcKcvnDYsln0wi8tz-LS_YzZ0x9FwQPeWoeQIPslIcfZ_rx5hdRXbaJESjyOl4RBIwHQutMcQp-frlBv6Bhv7qJc72dkxVds2yCzG8w_Qk8EsPl8L_Q8RE=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHJNK6udwSk_6juzloBhem_FucJpnHusQwZmqJ22mlAXNoJib4KdGXaF7T6KN-etYm1xve1TRwDX1klAl-OkmVYdeozJLy6a_C5I3DxMZ4hd8GYekjgDtHsI8c1pWSvAhxp7YWUFuhbraQIHE4H40E) [1, 5]. It models workflows as graphs, where each node represents a step (e.g., an LLM call or tool execution) and edges define the flow of control [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGstQdA0G2NyxK6qUIW_vpur4t1Hgws_-BBGtChcA31WONzZsUmD1HqLVLRLGVZ3FlfCXofkY_xZjvCItpgpMJRqxCujbqUqvWWNAQ5qnJlPuKYCinAK2N5TDbEwWRN45t0WFY47ISa94DIltgD3-iJFZxGK-ajW_QEZAsoEWP3qrYHRSS3Dwl7-SOxDXM-WAuTMcz6Qg==) [1, 5]. This graph-based approach is particularly beneficial for complex, stateful workflows that require clear visibility and control over an agent's reasoning process [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGstQdA0G2NyxK6qUIW_vpur4t1Hgws_-BBGtChcA31WONzZsUmD1HqLVLRLGVZ3FlfCXofkY_xZjvCItpgpMJRqxCujbqUqvWWNAQ5qnJlPuKYCinAK2N5TDbEwWRN45t0WFY47ISa94DIltgD3-iJFZxGK-ajW_QEZAsoEWP3qrYHRSS3Dwl7-SOxDXM-WAuTMcz6Qg==) [1].

In the financial services sector, AI, including LLMs like Gemini, is increasingly integrated to transform operations and decision-making [aalpha](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG63zq4norX-hlgO3Xb9KOeim62y6DNLqzCp2b-kxhhEEPd7yHmnbR-NlPJEZnqkbOtT-DB36YnOxpRhiB3ZAuU5FIFrvELzb8DMmeTNZsGZ7XZH4r-zQndOXKjCyCKls2NMvyWCM3_Gx1m9iXhpNyb4yoSxpIX). LangGraph, often used with LangChain, provides the framework for building AI agent workflows, enabling more flexible and efficient AI application management in this domain [gocodeo](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE0tO3B64jjzugo1EGYjrFiAIgrtO-gLT9O49KSX-xb_GOcYNDZzpn7gDdCXcxppn8xvkJgcahpiFi5NbXa0jbVlvGO4hiB4oCXkmVfowqyOzDFVd5pjgaJbvnffo9sZsAcM4H6i4u83us4OoNZ5IjTZGOOC-q8Kt_ngn2XuWQI_5FSowEULPcNH3HWkFizOakbwhj93Q_hJ20=) [coursera](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFDtU8Sv7YYpO-kkZhqHsI2wdMZhD3oA953Hc_gVCGOJrZ98h6PkoKj_QH3zCpHwmnxX2bifYj19Etm8rVKa4cfJXWmxJbNJX5SVPkYE_SzFNx1zBG9XWMmRv7P19l5XGv7z7pNnw==).

Here are detailed use cases that can leverage LangGraph with Google Gemini models in financial services:

### Core Concepts Enabling Financial Use Cases

LangGraph's design facilitates sophisticated AI applications through its fundamental components:
*   **State:** A shared data structure (often a TypedDict or Pydantic BaseModel) that maintains the application's current snapshot, persisting across different nodes in a workflow [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGp1wwnv1OYMUuBXqO7VKXvq3dGMf0knV6Qhl8apFr1y6sTCkwmlA9YereGAQvr6cfd5DkSqePZJZCJXnoNECu1I0e2LsozuOxNPw8f0nxS_PVLeK3u08M_zyHh-sYJsSqWG38WWIEWRsJp6VV0JawH_oQRJJo=) [1, 4]. This is crucial for maintaining context in long-running financial processes.
*   **Nodes:** These encapsulate the logic of individual agents or steps within the workflow. Nodes receive the current State as input, perform computations (such as LLM calls or tool invocations), and return an updated State [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGp1wwnv1OYMUuBXqO7VKXvq3dGMf0knV6Qhl8apFr1y6sTCkwmlA9YereGAQvr6cfd5DkSqePZJZCJXnoNECu1I0e2LsozuOxNPw8f0nxS_PVLeK3u08M_zyHh-sYJsSqWG38WWIEWRsJp6VV0JawH_oQRJJo=) [1, 4].
*   **Edges:** Edges dictate the flow of control, defining which node executes next based on the current State. This allows for both fixed transitions and dynamic, conditional logic [1, 2, 4]. This enables complex decision trees common in finance.

These components enable several powerful architectural patterns for financial applications:
*   **AI Agents:** LangGraph, combined with Gemini models, forms a strong foundation for building AI agents capable of perceiving their environment, making decisions, and executing actions to achieve specific goals [googleblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGstQdA0G2NyxK6qUIW_vpur4t1Hgws_-BBGtChcA31WONzZsUmD1HqLVLRLGVZ3FlfCXofkY_xZjvCItpgpMJRqxCujbqUqvWWNAQ5qnJlPuKYCinAK2N5TDbEwWRN45t0WFY47ISa94DIltgD3-iJFZxGK-ajW_QEZAsoEWP3qrYHRSS3Dwl7-SOxDXM-WAuTMcz6Qg==) [ubos](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGx1RAAvUKSuVJjH9gfvFtjmKgvnKWlhGiaXXyqDvt6NJZoNo47sjt7HolaUg9IBbBF-sB0fycf9vz2s4tTw6k7Vgy6D_Q6jFUewNO8MT88l-1VgBiY5JAgcI-uE36xmq0pU3n-Og2MEXjir-l1TdVI1sgMHiIeTsriwQBD0dma--Hk4psXR5y5rYui2YOlEQzTqmumGdDo8YKz) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHOa4cCbkDSMCEZpMRXQ15l0f_zal1Cz7G2WC-GZYKXXk5XQw0KsHooVpYpMFeTlccvQ_yrCMmORnWW2Qu1U3ftX3VBbdA6eafpGWfsC_M3KULUR5Bc8iD90k1wC5uKHgszYq1pONz1CdT-IhvmREx68FvnpAQ=) [1].
*   **Stateful Workflows:** LangGraph and Gemini models structure AI reasoning into stateful workflows, allowing an incoming query to progress through a series of nodes for tasks like routing, analysis, research, response generation, and validation [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpv5J4jgfQ2fp5z59nugTSmEQDrRkEyJ6F07UAqmx6r_FwERzTNWb_FAmeyzrntLYgzAXy3nD9npmGvJs4SFgZ54VdOtTzXn1STp2UQtUWzOVfKeNYeHOKrTlBEbr2HyksJuEx2_YXF9ZWLEvjZvmXpk9G2tAZDgtQ2A_f78QZoVeBEOSg5clwMqMXvKlXaDeV5_M5764pYwhSOcgxWL-HYnovn3SFkv7V846Lh0O6Gw3xXPnI1uRfSOtYbOPxE4RVtMCeb18=) [ubos](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGx1RAAvUKSuVJjH9gfvFtjmKgvnKWlhGiaXXyqDvt6NJZoNo47sjt7HolaUg9IBbBF-sB0fycf9vz2s4tTw6k7Vgy6D_Q6jFUewNO8MT88l-1VgBiY5JAgcI-uE36xmq0pU3n-Og2MEXjir-l1TdVI1sgMHiIeTsriwQBD0dma--Hk4psXR5y5rYui2YOlEQzTqmumGdDo8YKz) [2, 3].
*   **ReAct Agents:** LangGraph is particularly well-suited for constructing ReAct (Reasoning and Acting) Agents, which combine LLM reasoning with action execution, iteratively thinking, using tools, and acting on observations to dynamically adapt and achieve user goals [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGk3U4LFwGzRE509Ya98ghoYuI7ooo5planG5atbjq6P62m0swaoH1mspiuBkTOnCo3p3F09OtvOr8TBtjcoK_fTskXnk1ccKtpDDeU74yjvCiNz-WzFL2VWHl9Bk6K7dIFZxaPhYFThyzgCxeP5yjsr6I=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHOa4cCbkDSMCEZpMRXQ15l0f_zal1Cz7G2WC-GZYKXXk5XQw0KsHooVpYpMFeTlccvQ_yrCMmORnWW2Qu1U3ftX3VBbdA6eafpGWfsC_M3KULUR5Bc8iD90k1wC5uKHgszYq1pONz1CdT-IhvmREx68FvnpAQ=) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [huggingface](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH69nCSvxyR6rjSd4enlgbupDmKMNBZ4tFFf_WDNomOdWCxP32s1BltCF_8U4maHtyfoJcL2tSPl5gdaNMulHv9-DQR55Pz1Zuvfa8EDlHzJp27VZCmNwQet1LPRLKkcGmuAGg_fzShLqonlmnhZGcXfDNK-uHDOoF6ajDyBE0YKYU=) [4, 9].
*   **Multi-Agent Systems:** LangGraph simplifies the development of multi-agent systems by enabling the definition of agents with specific roles and goals, and then assigning tasks to them [1]. This is highly relevant for complex financial workflows.

### Key Use Cases in Financial Services

LangGraph with Google Gemini models can be applied to a wide array of complex scenarios within the financial sector:

*   **Financial Data Analysis and Trading:**
    *   LangGraph can orchestrate agents to analyze large financial datasets and execute trades based on predefined criteria [coursera](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFDtU8Sv7YYpO-kkZhqHsI2wdMZhD3oA953Hc_gVCGOJrZ98h6PkoKj_QH3zCpHwmnxX2bifYj19Etm8rVKa4cfJXWmxJbNJX5SVPkYE_SzFNx1zBG9XWMmRv7P19l5XGv7z7pNnw==) [coursera](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGAjhC6WZz8iCs8ijFyRNH2gqyR7pfqAbS7hwiQqb9TQvN-mx3XSeYA_d9CTPxJ1-qfj-y9lXFlKlDL50kpFr33HL1zhj-KBwm2pnBsOnnBgsa6uNd2Z8DcfQ39e5THC-HU8Z8E8mc=).
    *   Agents can identify trends and patterns in real-time market data [coursera](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFDtU8Sv7YYpO-kkZhqHsI2wdMZhD3oA953Hc_gVCGOJrZ98h6PkoKj_QH3zCpHwmnxX2bifYj19Etm8rVKa4cfJXWmxJbNJX5SVPkYE_SzFNx1zBG9XWMmRv7P19l5XGv7z7pNnw==) [coursera](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGAjhC6WZz8iCs8ijFyRNH2gqyR7pfqAbS7hwiQqb9TQvN-mx3XSeYA_d9CTPxJ1-qfj-y9lXFlKlDL50kpFr33HL1zhj-KBwm2pnBsOnnBgsa6uNd2Z8DcfQ39e5THC-HU8Z8E8mc=).
    *   Gemini models can gauge market sentiment by analyzing news, social media, and financial reports to anticipate market movements [coursera](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFDtU8Sv7YYpO-kkZhqHsI2wdMZhD3oA953Hc_gVCGOJrZ98h6PkoKj_QH3zCpHwmnxX2bifYj19Etm8rVKa4cfJXWmxJbNJX5SVPkYE_SzFNx1zBG9XWMmRv7P19l5XGv7z7pNnw==) [coursera](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGAjhC6WZz8iCs8ijFyRNH2gqyR7pfqAbS7hwiQqb9TQvN-mx3XSeYA_d9CTPxJ1-qfj-y9lXFlKlDL50kpFr33HL1zhj-KBwm2pnBsOnnBgsa6uNd2Z8DcfQ39e5THC-HU8Z8E8mc=).
    *   AI-driven insights can lead to better investment decisions [analyticsvidhya](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEK9dxXEvu9NDeG3dwkAkmHaOivSdfXE8jIuyCcPL5rJg0SBfaRyKDirvcqmQr-_FronBbrDdmoc2y4Sm1UoZ8EVdIhXwtzL6MNrukPX-tzfz6qNjt9bMOIAH3sNYJmHYFV9-JpGvS51StHHXcfZHxmz_0K7Oy5HYPZHUFvmFqtxmA-9Yyjs-_KfsM=) [spendesk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH724jeNIi_tHhYgF9zAF6_geiUXDK6N5nMXl__rQt4Ycbu78Pa60S5EmqXOiuL4ZH6LvOlAUdSjsrk_1Lp-2KIA5ni0n_ODzg46l2QCdObfOG4DxhgQRMBSLNBubH27X2RFxplI9lKbbmrHaJBeJzY5vYAdrM1OA==).
    *   LangGraph enables algorithmic trading with explainable AI for transparency and risk management [spendesk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH724jeNIi_tHhYgF9zAF6_geiUXDK6N5nMXl__rQt4Ycbu78Pa60S5EmqXOiuL4ZH6LvOlAUdSjsrk_1Lp-2KIA5ni0n_ODzg46l2QCdObfOG4DxhgQRMBSLNBubH27X2RFxplI9lKbbmrHaJBeJzY5vYAdrM1OA==). Gemini 2.5 Pro's "Deep Think" capabilities can enhance reasoning for complex trading strategies [revolgy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE0tO3B64jjzugo1EGYjrFiAIgrtO-gLT9O49KSX-xb_GOcYNDZzpn7gDdCXcxppn8xvkJgcahpiFi5NbXa0jbVlvGO4hiB4oCXkmVfowqyOzDFVd5pjgaJbvnffo9sZsAcM4H6i4u83us4OoNZ5IjTZGOOC-q8Kt_ngn2XuWQI_5FSowEULPcNH3HWkFizOakbwhj93Q_hJ20=2).

*   **Customer Service and Robo-Advisors:**
    *   AI-driven chatbots can provide instant, personalized financial advice and customer support [arya](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFXzvRD8N3oAv4CT5T5H4ujvz3TD4COjJipGyhUX_6KkqqNbLbXbM4tXiYpX40xU0adK56TZ7RoNA3Z2qatTXi65uiQbVZeaDr1Md_qZZojiglO9fxA5AP3RwYTxNoRBoQTAdabm5w8V4KCj4DSaPnyiToOELY2Nm3SMcpXXi5Tv-nSXOp8eOg=).
    *   Robo-advisors can assess risk, adjust portfolios, and explain decisions, with LangGraph managing the state and flow of these interactions [so-development](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFnUAO6EyPnQ1lr6V2WWOGfiJKLa1Ib3Hn03O5P8aPsvJKdkZWUJCgYTjvV1wFb0SQjSbX_EX3gTNGSdzoli50xF5ZwWDoO_0ALjtBcAAwOQtbqzi4ffqxerg9mZhJapt2GTHHviXSbR3Dyvf2EYaGG94W_Z54Yl0QljttPy1PiKbZat9jY1vn5PfknIGZB4A1UcuQ3L_xHMxATRwEY).
    *   These systems can create personalized customer journeys [spendesk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH724jeNIi_tHhYgF9zAF6_geiUXDK6N5nMXl__rQt4Ycbu78Pa60S5EmqXOiuL4ZH6LvOlAUdSjsrk_1Lp-2KIA5ni0n_ODzg46l2QCdObfOG4DxhgQRMBSLNBubH27X2RFxplI9lKbbmrHaJBeJzY5vYAdrM1OA==).

*   **Risk Management and Fraud Detection:**
    *   LangGraph facilitates predictive risk management by orchestrating agents that analyze vast amounts of market data and economic indicators [spendesk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH724jeNIi_tHhYgF9zAF6_geiUXDK6N5nMXl__rQt4Ycbu78Pa60S5EmqXOiuL4ZH6LvOlAUdSjsrk_1Lp-2KIA5ni0n_ODzg46l2QCdObfOG4DxhgQRMBSLNBubH27X2RFxplI9lKbbmrHaJBeJzY5vYAdrM1OA==) [coursera](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGAjhC6WZz8iCs8ijFyRNH2gqyR7pfqAbS7hwiQqb9TQvN-mx3XSeYA_d9CTPxJ1-qfj-y9lXFlKlDL50kpFr33HL1zhj-KBwm2pnBsOnnBgsa6uNd2Z8DcfQ39e5THC-HU8Z8E8mc=).
    *   It can be used to detect and prevent fraudulent activities [devoteam](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFOldk9T21-SFTcGTTNlrXdkmo0Fk-Plxl7qCKHyQi2a0NwMBbxw_fRITPpNa9uViv6LXNdlMqJC5TT9Opun9m5vA50hQ3C8sdO59fBTj4ivMTGUVWIvqFm3pstxcOWibAIEaxc9Wad8OyaQQAtJ5z1bJrh5toqo7u9) [lyzr](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGxKlyOPucCke49bqg1Ayk67TXlToFM4W5bZ_wPSisia65miqixE-416hw3dU7A_NAhbsar33v6WrSvqY1XnyPvPCXCTDC2BNhdPlQ7v1AXjFM-6LO0DsPerKhnUYfuA9NY).
    *   Compliance monitoring, tracking insider trades, and reporting suspicious activity can be streamlined [so-development](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFnUAO6EyPnQ1lr6V2WWOGfiJKLa1Ib3Hn03O5P8aPsvJKdkZWUJCgYTjvV1wFb0SQjSbX_EX3gTNGSdzoli50xF5ZwWDoO_0ALjtBcAAwOQtbqzi4ffqxerg9mZhJapt2GTHHviXSbR3Dyvf2EYaGG94W_Z54Yl0QljttPy1PiKbZat9jY1vn5PfknIGZB4A1UcuQ3L_xHMxATRwEY).
    *   AI-driven credit scoring and underwriting can also leverage these capabilities [aalpha](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG63zq4norX-hlgO3Xb9KOeim62y6DNLqzCp2b-kxhhEEPd7yHmnbR-NlPJEZnqkbOtT-DB36YnOxpRhiB3ZAuU5FIFrvELzb8DMmeTNZsGZ7XZH4r-zQndOXKjCyCKls2NMvyWCM3_Gx1m9iXhpNyb4yoSxpIX).

*   **Automation and Efficiency:**
    *   Automating data processing and financial operations, including data entry, removing duplicates, and ensuring accuracy [lyzr](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGxKlyOPucCke49bqg1Ayk67TXlToFM4W5bZ_wPSisia65miqixE-416hw3dU7A_NAhbsar33v6WrSvqY1XnyPvPCXCTDC2BNhdPlQ7v1AXjFM-6LO0DsPerKhnUYfuA9NY) [arya](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFXzvRD8N3oAv4CT5T5H4ujvz3TD4COjJipGyhUX_6KkqqNbLbXbM4tXiYpX40xU0adK56TZ7RoNA3Z2qatTXi65uiQbVZeaDr1Md_qZZojiglO9fxA5AP3RwYTxNoRBoQTAdabm5w8V4KCj4DSaPnyiToOELY2Nm3SMcpXXi5Tv-nSXOp8eOg=) [f9finance](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9rW3xuvrHfSrhP9k9cG8SS7b13qkkJf_TrFjI-_ZSDvVLoXDJH6Q17drsoLCl2_-c9mI5mUE3C7MiOfQ7UlaFT5TYV1BVBRqRqK3DpWpQPoGbzhpGOsB13I97J7TGIjDgIQTULj561kSbXuoAHC0=).
    *   Automating financial reporting and generating comprehensive reports using preset templates [arya](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFXzvRD8N3oAv4CT5T5H4ujvz3TD4COjJipGyhUX_6KkqqNbLbXbM4tXiYpX40xU0adK56TZ7RoNA3Z2qatTXi65uiQbVZeaDr1Md_qZZojiglO9fxA5AP3RwYTxNoRBoQTAdabm5w8V4KCj4DSaPnyiToOELY2Nm3SMcpXXi5Tv-nSXOp8eOg=) [f9finance](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9rW3xuvrHfSrhP9k9cG8SS7b13qkkJf_TrFjI-_ZSDvVLoXDJH6Q17drsoLCl2_-c9mI5mUE3C7MiOfQ7UlaFT5TYV1BVBRqRqK3DpWpQPoGbzhpGOsB13I97J7TGIjDgIQTULj561kSbXuoAHC0=).
    *   Streamlining compliance processes and improving regulatory reporting [devoteam](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFOldk9T21-SFTcGTTNlrXdkmo0Fk-Plxl7qCKHyQi2a0NwMBbxw_fRITPpNa9uViv6LXNdlMqJC5TT9Opun9m5vA50hQ3C8sdO59fBTj4ivMTGUVWIvqFm3pstxcOWibAIEaxc9Wad8OyaQQAtJ5z1bJrh5toqo7u9) [spendesk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH724jeNIi_tHhYgF9zAF6_geiUXDK6N5nMXl__rQt4Ycbu78Pa60S5EmqXOiuL4ZH6LvOlAUdSjsrk_1Lp-2KIA5ni0n_ODzg46l2QCdObfOG4DxhgQRMBSLNBubH27X2RFxplI9lKbbmrHaJBeJzY5vYAdrM1OA==).
    *   Automated bookkeeping and invoicing [aalpha](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG63zq4norX-hlgO3Xb9KOeim62y6DNLqzCp2b-kxhhEEPd7yHmnbR-NlPJEZnqkbOtT-DB36YnOxpRhiB3ZAuU5FIFrvELzb8DMmeTNZsGZ7XZH4r-zQndOXKjCyCKls2NMvyWCM3_Gx1m9iXhpNyb4yoSxpIX).

*   **Customer Onboarding and KYC/AML Processes:**
    *   AI agents can validate identity documents, cross-check user data against PEP/sanction lists, and flag high-risk profiles during onboarding [aalpha](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHoO5SMDDnUio3HUxRUoDNYbBe_I-ggIydazuWCBW-0o4_icyOXbdhxyBlgqiJhLpFemv97Hjngwtfu7jB8ujXD4r38QyMadVdPhJ4FSJAZMwEHZmRVnnh6iA4APkLHDsW-xulVWVwsNJJPRpgotudosrGGjtH-aA==) [lyzr](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEbwQnK_j167d88_A-u3Rd2DbwDwoT55jprxUT8oBXaC_RnefROUQml3hdUvttujzn9RVCK_uw_Inpf7Mavrr63a-VICJzO4IRASWkbHC2SUCUdtMNlG-efPXZTCzC7Mg2f).
    *   LangGraph's multi-stage processing and conditional edges are ideal for guiding customers through complex verification steps based on their risk profile.

*   **Investment Research:**
    *   AI agents can analyze vast amounts of financial data to identify investment opportunities [researchgate](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFmYsmRpRja7n97iDNVmWzihx65Ar43cDS-xt99BjrMzPWK-iLC9wSpopZd4gh7Aknd_sdAPYVuUBHjtcn1fzmAZ8-fgNj13aH19RyLNnrQvubDg5UtsZj35lTC-IB6io-zK3rtxAlR11HdKEjbopj1U6kIsC-lW15klqa2iggqV2wjZUr4VBmsTOy8pa19lp0eWP0G3nMVPa5HwftMfxejaazMXfs5DhKO5MzBM1GmZmlI3dnnPuVfmT_PXBvZuQg9civiRGkZKfj-OrPfMtg_DDlSwge7vQSKRSJj5J659XBIc0s=).
    *   LangGraph enables the creation of systems that scale insight generation and make services available to more clients [youtube](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGNcy9DD3ULlDV0tr1HDP0_3aLJZsTCz_H6e5qC7atxzKnTNKxhoB4YBwLLbypGnTf8bSZAWhNpEXRLPOJTBVj8xKwS_w-z31lIqaU1Svp3JOMpqHItJoT6Am10p6KyAn0yXpaQDFY=).
    *   Gemini can aid in interpreting financial statements by automating data extraction and generating concise summaries [skillfine](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEA0RIlJD8Rqt8oEhGaL4Da4O1DKWNhfjLbU8yl0eBvwXEPjBkiETfCrFqLFPOmHzASUsclnowuX3Fz7PgXAlYvwGUM4DSqjKqzCDPcXvuH0GEqy3s4Q0cayjIHyb85EW_qjNzFrpdf7IGIQHoo0glrLMRsWpg7rQ==].

*   **Multimodal Agents:**
    *   LangGraph can be combined with Gemini to create multimodal agents capable of detecting objects in images, audio, and video [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE0tO3B64jjzugo1EGYjrFiAIgrtO-gLT9O49KSX-xb_GOcYNDZzpn7gDdCXcxppn8xvkJgcahpiFi5NbXa0jbVlvGO4hiB4oCXkmVfowqyOzDFVd5pjgaJbvnffo9sZsAcM4H6i4u83us4OoNZ5IjTZGOOC-q8Kt_ngn2XuWQI_5FSowEULPcNH3HWkFizOakbwhj93Q_hJ20=4) [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH8rvFr8qV0VwKCQyFWZgRcqVhTUpVGgfhC356sydoILM4WhYCXjUseegWVBKYke3AHsre7sLUKVMuq1F9t87qNgkvhxQm30p5dwUtjOepqNmH5tE6W-14xLFBYR9Bm4umT-loqzULot1NEngj-96G-QnLT5iWjPJ-jgQuADT_xeGxutV4lNBHJMhTCl5Z1sz6p1ktLGIe98iVrxp_nBLBX0jjZ-6fmFlO8JOAjjxaf22w=). This is valuable for financial applications like document verification (e.g., scanning IDs for KYC) or analyzing multimedia for compliance.

*   **Tool Integration:**
    *   Gemini models' function calling capabilities allow LangGraph agents to interact seamlessly with external tools, APIs, and data sources, enabling them to perform real-world actions. This includes integrating tools for web scraping, search, and specialized APIs for domains like finance (e.g., Yahoo Finance) [marktechpost](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE0tO3B64jjzugo1EGYjrFiAIgrtO-gLT9O49KSX-xb_GOcYNDZzpn7gDdCXcxppn8xvkJgcahpiFi5NbXa0jbVlvGO4hiB4oCXkmVfowqyOzDFVd5pjgaJbvnffo9sZsAcM4H6i4u83us4OoNZ5IjTZGOOC-q8Kt_ngn2XuWQI_5FSowEULPcNH3HWkFizOakbwhj93Q_hJ20=2) [creolestudios](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH8rvFr8qV0VwKCQyFWZgRcqVhTUpVGgfhC356sydoILM4WhYCXjUseegWVBKYke3AHsre7sLUKVMuq1F9t87qNgkvhxQm30p5dwUtjOepqNmH5tE6W-14xLFBYR9Bm4umT-loqzULot1NEngj-96G-QnLT5iWjPJ-jgQuADT_xeGxutV4lNBHJMhTCl5Z1sz6p1ktLGIe98iVrxp_nBLBX0jjZ-6fmFlO8JOAjjxaf22w=).

### Practical Implementation Details

When building LangGraph agents with Gemini for financial services, several practical aspects are key:
*   **Node Configurations:** Common node types include `call_tool` (often using LangGraph's prebuilt `ToolNode`) for executing tool methods (e.g., financial API calls), and `call_model` for invoking the Gemini model for reasoning or generation [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGp1wwnv1OYMUuBXqO7VKXvq3dGMf0knV6Qhl8apFr1y6sTCkwmlA9YereGAQvr6cfd5DkSqePZJZCJXnoNECu1I0e2LsozuOxNPw8f0nxS_PVLeK3u08M_zyHh-sYJsSqWG38WWIEWRsJp6VV0JawH_oQRJJo=) [1].
*   **Edge Configurations:** Edges define transitions. A `should_continue` edge can decide whether to call a tool or the model [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [philschmid](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGp1wwnv1OYMUuBXqO7VKXvq3dGMf0knV6Qhl8apFr1y6sTCkwmlA9YereGAQvr6cfd5DkSqePZJZCJXnoNECu1I0e2LsozuOxNPw8f0nxS_PVLeK3u08M_zyHh-sYJsSqWG38WWIEWRsJp6VV0JawH_oQRJJo=) [1, 4]. Conditional edges enable dynamic workflow based on node outcomes, crucial for adapting to financial market changes or customer profiles [2, 8].
*   **State Management:** LangGraph provides helpers like `add_messages` for efficiently updating message lists within the agent's state [google](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=) [1].
*   **Human-in-the-Loop:** LangGraph supports incorporating human intervention and oversight into agent workflows [github](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyI2nJk5jlT8POmfXbVJscryUwfDqPkWpU-g8ZhDGyhz57FbsXFej79qMVodFS73Fm1CYCKL6uf_stobqMIyOwedD3d1K0SlhRkic9Fbt8wtaNoOjVcKMeYPW5lNIv0C4GZ1bIIgWQ17aC71QoCrRudXo=2) [11], which is crucial for complex or sensitive financial applications requiring expert review.

### Challenges and Considerations

While LangGraph and Gemini offer significant benefits, it's important to address challenges in financial services:
*   **Data Privacy and Security:** Protecting sensitive customer and financial data is critical [12]. Gemini models becoming available on-premise (Google Distributed Cloud, public preview Q3 2025) can help address data residency and security concerns [capacitymedia](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE0tO3B64jjzugo1EGYjrFiAIgrtO-gLT9O49KSX-xb_GOcYNDZzpn7gDdCXcxppn8xvkJgcahpiFi5NbXa0jbVlvGO4hiB4oCXkmVfowqyOzDFVd5pjgaJbvnffo9sZsAcM4H6i4u83us4OoNZ5IjTZGOOC-q8Kt_ngn2XuWQI_5FSowEULPcNH3HWkFizOakbwhj93Q_hJ20=0) [googlecloudblog](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH8rvFr8qV0VwKCQyFWZgRcqVhTUpVGgfhC356sydoILM4WhYCXjUseegWVBKYke3AHsre7sLUKVMuq1F9t87qNgkvhxQm30p5dwUtjOepqNmH5tE6W-14xLFBYR9Bm4umT-loqzULot1NEngj-96G-QnLT5iWjPJ-jgQuADT_xeGxutV4lNBHJMhTCl5Z1sz6p1ktLGIe98iVrxp_nBLBX0jjZ-6fmFlO8JOAjjxaf22w=).
*   **Regulatory Compliance:** Adhering to strict financial regulations is essential [spendesk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH724jeNIi_tHhYgF9zAF6_geiUXDK6N5nMXl__rQt4Ycbu78Pa60S5EmqXOiuL4ZH6LvOlAUdSjsrk_1Lp-2KIA5ni0n_ODzg46l2QCdObfOG4DxhgQRMBSLNBubH27X2RFxplI9lKbbmrHaJBeJzY5vYAdrM1OA==).
*   **Model Accuracy and Hallucinations:** Mitigating the risk of LLMs producing incorrect information is paramount in finance [12].
*   **Integration Complexity:** Overcoming the complexity of integrating AI agents into existing legacy systems can be a barrier [aalpha](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG63zq4norX-hlgO3Xb9KOeim62y6DNLqzCp2b-kxhhEEPd7yHmnbR-NlPJEZnqkbOtT-DB36YnOxpRhiB3ZAuU5FIFrvELzb8DMmeTNZsGZ7XZH4r-zQndOXKjCyCKls2NMvyWCM3_Gx1m9iXhpNyb4yoSxpIX) [lyzr](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEbwQnK_j167d88_A-u3Rd2DbwDwoT55jprxUT8oBXaC_RnefROUQml3hdUvttujzn9RVCK_uw_Inpf7Mavrr63a-VICJzO4IRASWkbHC2SUCUdtMNlG-efPXZTCzC7Mg2f).

In summary, LangGraph and Google Gemini models are poised to significantly impact financial services by enabling the development of sophisticated, stateful AI agents. These agents can automate complex workflows, provide real-time insights, enhance customer experiences, and improve risk management, driving productivity and cost savings across the industry [promevo](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFJj7IDXYi_4Ro-_K-bTL50aBlS_4O2vSWepi57j3eJR6bKRwbyOesshq7wvzrb9KRNj_HMCAcmqOA0EuAOScoA7N5bUVK1lHbtnK37WDtfYv2E3A7BBY5i2ml4O8yvorKsnu6repi6rkvsidaSE36CBA==).